In [1]:
from utils.functions import *
import pandas as pd

In [2]:
df = pd.DataFrame()

In [3]:
for i in range(2011, 2021):
    if i <=2012:
        link = 'data/' + str(i) + '.xls'
    else:
        link = 'data/' + str(i) + '.xlsx'
    df_temp = pd.read_excel(link)
    df = df.append(df_temp, sort=False)

In [4]:
df = df.reset_index()

In [5]:
df = df[df.Date.notnull()]

In [6]:
df['Date'] = df.apply(lambda row: datetime.strptime(str(row['Date']), "%Y-%m-%d %H:%M:%S").strftime("%Y/%m/%d"), axis=1)

In [7]:
df.reset_index(inplace=True)

In [8]:
del (df['index'])

In [9]:
df = df[df.Comment == 'Completed']

In [10]:
df.loc[:,'best_of_5'] = (df['Best of'] ==5).astype(int)
df['W3'] = pd.to_numeric(df['W3'], errors='coerce')
df['L3'] = pd.to_numeric(df['L3'], errors='coerce')

In [11]:
df = df.fillna(0)

In [12]:
print(df.dtypes)

level_0         int64
ATP             int64
Location       object
Tournament     object
Date           object
Series         object
Court          object
Surface        object
Round          object
Best of         int64
Winner         object
Loser          object
WRank         float64
LRank         float64
WPts          float64
LPts          float64
W1            float64
L1            float64
W2            float64
L2            float64
W3            float64
L3            float64
W4            float64
L4            float64
W5            float64
L5            float64
Wsets         float64
Lsets         float64
Comment        object
B365W         float64
B365L         float64
EXW            object
EXL           float64
LBW           float64
LBL           float64
PSW           float64
PSL           float64
SJW           float64
SJL           float64
MaxW          float64
MaxL          float64
AvgW          float64
AvgL          float64
best_of_5       int32
dtype: object


In [13]:
df_combined = df[['Tournament',  'Date', 'Surface', 'Round']].copy()

In [14]:
df_combined.loc[:,'player_0'] = df.apply(lambda row: get_player_1_name(row['Winner'], row['WRank'], row['Loser'], row['LRank']), axis=1)
df_combined.loc[:,'player_0_rank'] = df.apply(lambda row: get_player_1_rank(row['WRank'], row['LRank']), axis=1)
df_combined.loc[:,'player_0_odd'] = df.apply(lambda row: get_player_1_odd(row['B365W'], row['WRank'], row['B365L'], row['LRank']), axis=1)

In [15]:
df_combined.loc[:,'player_1'] = df.apply(lambda row: get_player_2_name(row['Winner'], row['WRank'], row['Loser'], row['LRank']), axis=1)
df_combined.loc[:,'player_1_rank'] = df.apply(lambda row: get_player_2_rank(row['WRank'], row['LRank']), axis=1)
df_combined.loc[:,'player_1_odd'] = df.apply(lambda row: get_player_2_odd(row['B365W'], row['WRank'], row['B365L'], row['LRank']), axis=1)

In [16]:
df_combined.loc[:,'outcome'] = df.apply(lambda row: outcome(row['WRank'], row['LRank']), axis=1)

In [17]:
df_combined = df_combined[df_combined.Tournament == 'Australian Open']

In [18]:
df_combined = df_combined[df_combined.Date > '2010/01/01']

In [19]:
df_combined = create_features(df_combined, df)

Creating Player Career Stats All Surface
Creating Player Career Stats on Hard Court
Creating Player Career Stats All Surface Last 60 Weeks
Creating Player Career Stats on Hard Court Last 100 Weeks
Creating Player Head to Head Career Stats All Surface
Creating Player Head to Head Career Stats On Hard Court
Creating Difference Variables


In [20]:
print(df_combined.columns)

Index(['Tournament', 'Date', 'Surface', 'Round', 'player_0', 'player_0_rank',
       'player_0_odd', 'player_1', 'player_1_rank', 'player_1_odd', 'outcome',
       'player_0_match_win_percent', 'player_1_match_win_percent',
       'player_0_games_win_percent', 'player_1_games_win_percent',
       'player_0_5_set_match_win_percent', 'player_1_5_set_match_win_percent',
       'player_0_close_sets_percent', 'player_1_close_sets_percent',
       'player_0_match_win_percent_hard', 'player_1_match_win_percent_hard',
       'player_0_games_win_percent_hard', 'player_1_games_win_percent_hard',
       'player_0_5_set_match_win_percent_hard',
       'player_1_5_set_match_win_percent_hard',
       'player_0_close_sets_percent_hard', 'player_1_close_sets_percent_hard',
       'player_0_match_win_percent_60', 'player_1_match_win_percent_60',
       'player_0_games_win_percent_60', 'player_1_games_win_percent_60',
       'player_0_5_set_match_win_percent_60',
       'player_1_5_set_match_win_percent

In [21]:
df_combined.head()

,Tournament,Date,Surface,Round,player_0,player_0_rank,player_0_odd,player_1,player_1_rank,player_1_odd,...,diff_5_set_match_win_percent_60,diff_close_sets_percent_60,diff_match_win_percent_hard_100,diff_games_win_percent_hard_100,diff_5_set_match_win_percent_hard_100,diff_close_sets_percent_hard_100,diff_match_win_percent_hh,diff_games_win_percent_hh,diff_match_win_percent_hard_hh,diff_games_win_percent_hard_hh
147,Australian Open,2011/01/17,Hard,1st Round,Haase R.,62.0,1.10,Berlocq C.,67.0,6.50,...,0.0,0.400000,0.600000,0.089662,0.0,0.400000,0.0,0.0,0.0,0.0
148,Australian Open,2011/01/17,Hard,1st Round,Monfils G.,12.0,1.14,De Bakker T.,47.0,5.00,...,0.0,-0.500000,0.000000,-0.428571,0.0,-0.500000,0.0,0.0,0.0,0.0
149,Australian Open,2011/01/17,Hard,1st Round,Fognini F.,58.0,2.62,Nishikori K.,82.0,1.44,...,0.0,-0.277778,-0.666667,-0.150641,0.0,-0.277778,0.0,0.0,0.0,0.0
150,Australian Open,2011/01/17,Hard,1st Round,Przysiezny M.,84.0,1.66,Kunitsyn I.,99.0,2.10,...,0.0,0.333333,0.000000,0.423077,0.0,0.333333,0.0,0.0,0.0,0.0
151,Australian Open,2011/01/17,Hard,1st Round,Cuevas P.,64.0,1.36,Gil F.,91.0,3.00,...,0.0,1.000000,0.000000,0.341872,0.0,1.000000,0.0,0.0,0.0,0.0


In [22]:
df.head()

,level_0,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,...,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,best_of_5
0,0,1,Brisbane,Brisbane International,2011/01/02,ATP250,Outdoor,Hard,1st Round,3,...,2.38,1.67,2.33,0.00,0.00,1.67,2.60,1.58,2.36,0
1,1,1,Brisbane,Brisbane International,2011/01/02,ATP250,Outdoor,Hard,1st Round,3,...,1.83,2.12,1.79,1.83,1.83,2.10,1.85,1.99,1.76,0
2,2,1,Brisbane,Brisbane International,2011/01/03,ATP250,Outdoor,Hard,1st Round,3,...,1.73,1.98,1.91,2.00,1.73,2.01,1.91,1.91,1.82,0
3,3,1,Brisbane,Brisbane International,2011/01/03,ATP250,Outdoor,Hard,1st Round,3,...,1.91,1.78,2.15,1.83,1.83,1.91,2.16,1.80,1.94,0
4,4,1,Brisbane,Brisbane International,2011/01/03,ATP250,Outdoor,Hard,1st Round,3,...,2.75,1.41,3.10,1.44,2.63,1.44,3.15,1.39,2.88,0


In [23]:
df_combined.to_csv('data/aus_open_matches_with_features.csv', index=False)

df.to_csv('data/combined_raw_data.csv', index=False)